# Imports and Setup
To begin, run the cell(s) below to install the necessary setup and imports.

In [ ]:
!sudo apt install python3.8

In [ ]:
!python3.8 --version

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [ ]:
!conda update -n base -c defaults conda --yes

In [6]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.8/site-packages"))

In [ ]:
import sys
sys.path

In [ ]:
!conda init zsh

In [ ]:
!conda create --name env python=3.8 --yes

In [10]:
!source /usr/local/bin/activate env

In [ ]:
%%bash
pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
pip install timm==0.4.12
pip install pickle5
pip install scikit-learn
pip install numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/pipline/

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py --help

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-model PDAN \
-num_channel 512 \
-lr 0.0002 \
-kernelsize 3 \
-APtype map \
-epoch 15 \
-batch_size 1 \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB \
-rgb_model_file P02T01C06

In [ ]:
!pip install ipywidgets

## Initializing Constants

In [ ]:
# INITIALIZING PATH VARIABLES (CHANGE HERE IF DIFFERENT DIRECTORY)

PATH_TO_MODEL_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/model/"
PATH_TO_VIDEO_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/video/"
PATH_TO_FEATURE_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/"

# R2

Data Exploration Section:
- Check out the different input videos (with UI)
- See playback of video

In [ ]:
import os
import ipywidgets as widgets

videoList = os.listdir(PATH_TO_VIDEO_FOLDER)

video = widgets.Dropdown(
    options = videoList,
    description='Video:',
    disabled=False,
)
video

In [ ]:
from IPython.display import Video

VIDEO_SELECTED = PATH_TO_VIDEO_FOLDER + video.value
Video(VIDEO_SELECTED)

# R3
Inference Section:
- Load a pretrained model (with UI)
- Choose input video (with UI)
- See inference result in the form out output video with captions that indicte the current detected activity in each video frame

In [ ]:
import os
import ipywidgets as widgets

modelList = os.listdir(PATH_TO_MODEL_FOLDER)

model = widgets.Dropdown(
    options = modelList,
    description='Model:',
    disabled=False,
)

box = widgets.Box(children = [model, video])
box

In [ ]:
#finalized chosen model and video directory

SELECTED_MODEL = PATH_TO_MODEL_FOLDER + model.value
SELECTED_VIDEO = PATH_TO_VIDEO_FOLDER + video.value
print(SELECTED_MODEL)
print(SELECTED_VIDEO)

# FEATURE EXTRACTION

TSU I3D Feature Extraction from video

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/

! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6 av==8.0.2

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/video_features

In [ ]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

# TODO: RE-ADJUST THE DIRECTORY TO BE TO FEATURE_FILE

# TODO: RE-ADJUST TRAIN_CUSTOM.PY

In [ ]:
# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/videos/sample_videos/sample_video_1.mp4']
# args.show_pred = True
# args.stack_size = 24
# args.step_size = 24
# args.extraction_fps = 25
args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
# args.streams = 'flow'

# Load the model
extractor = ExtractI3D(args)

# Extract features
for video_path in args.video_paths:
    # extract name of video from video path (and remove '.mp4' suffix)
    video_name = (video_path.split('/')[-1]).split('.')[0]
    print(video_name)
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    print(feature_dict.items())
    for k, v in feature_dict.items():
      print(k)
      print(v.shape)
      print(v)
      if k == "rgb":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/rgb/{video_name}_rgb', v)
      elif k == "flow":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/flow/{video_name}_flow', v)
      elif k == "fps":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/fps/{video_name}_fps', v)
      elif k == "timestamps_ms":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/timestamps_ms/{video_name}_timestamps_ms', v)
      else:
        raise NotImplementedError("Error: Unexpected item in feature_dict!")

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/rgb/sample_video_1_rgb.npy')
print(data)

# R4 

- Choose a dataset subfolder to use for training (with UI)
- Initialize a model with a network architecture configured in a separate .py file

In [ ]:
import os
import ipywidgets as widgets

featureFolder = os.listdir(PATH_TO_FEATURE_FOLDER)

feature = widgets.Dropdown(
    options = featureFolder,
    description='Feature:',
    disabled=False,
)
feature

# R4.3

- As a User, I want to have UI elements that adds a name to the model from R4.2, so that I can specify the name for this new model

In [ ]:
import os
import ipywidgets as widgets

input_text = widgets.Text(
    value='',
    placeholder='Enter Model name',
    description='Model name:',
    disabled=False
)

input_text

In [ ]:
print(input_text.value)
%cd /content/drive/MyDrive/Toyota_Smarthome-main/model/
!pwd
!ls
# !mv $input_text.value PDAN_TSU_RGB 
# !mv PDAN_TSU_RGB $input_text.value 
# !mv PDAN_TSU_RGB.value PDAN_TSU_RGB
# !mv PDAN_TSU_RGB.value PDAN_TSU_RGB
!ls

# R4.4 
- As a User, I want to set the batch_size and epochs with appropriate UI elements, so that I can configure the model prior to running a training sequence.



In [ ]:
import os
import ipywidgets as widgets

batch_size = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='batch_size:',
    disabled=False
)
epoch = widgets.BoundedIntText(
    value=1,
    min=1,
    max=2000,
    step=1,
    description='epoch:',
    disabled=False
)
modelbox = widgets.Box(children = [batch_size, epoch])
modelbox

In [ ]:
print(batch_size.value)
print(epoch.value)

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-model PDAN \
-num_channel 512 \
-lr 0.0002 \
-kernelsize 3 \
-APtype map \
-epoch $epoch.value \
-batch_size $batch_size.value \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB \
-rgb_model_file P02T01C06